<a href="https://colab.research.google.com/github/harjeet88/agentic_ai/blob/main/crew_ai/agentic_engg_team.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install required pacakges


In [1]:
!pip install -q -U crewai
!pip install -q -U duckduckgo-search
!pip install -q -U crewai_tools

### Required Packages

In [3]:
import os

from crewai import Agent, Task, Crew, Process, LLM
#from crewai_tools import tool

In [9]:
from crewai.tools import BaseTool

In [4]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
os.environ["GEMINI_API_KEY"] = GOOGLE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
if GOOGLE_API_KEY :
  print("GOOGLE_API_KEY is set")

GOOGLE_API_KEY is set


### set up the LLM

In [5]:
from crewai import Agent, LLM

gemini = LLM(
    api_key=os.getenv("GEMINI_API_KEY"),
    model="gemini/gemini-1.5-flash",
)

In [6]:
my_llm=gemini

### setup tools

In [13]:
!pip install --upgrade --quiet  langchain-community

In [18]:
engineering_lead = Agent(
    role = 'Engineering Lead for the engineering team, directing the work of the engineer',
    goal = '''Take the high level requirements described here and prepare a detailed design for the backend developer;
    everything should be in 1 python module; describe the function and method signatures in the module.
    The python module must be completely self-contained, and ready so that it can be tested or have a simple UI built for it.
    Here are the requirements: {requirements}
    The module should be named {module_name} and the class should be named {class_name}''',
    backstory = '''You're a seasoned engineering lead with a knack for writing clear and concise designs.''',
    llm = my_llm,
    verbose=True,
)

In [19]:
backend_engineer = Agent(
    role = 'Python Engineer who can write code to achieve the design described by the engineering lead',
    goal = '''Write a python module that implements the design described by the engineering lead, in order to achieve the requirements.
    The python module must be completely self-contained, and ready so that it can be tested or have a simple UI built for it.
    Here are the requirements: {requirements}
    The module should be named {module_name} and the class should be named {class_name}''',
    backstory = '''You're a seasoned python engineer with a knack for writing clean, efficient code.
    You follow the design instructions carefully.
    You produce 1 python module named {module_name} that implements the design and achieves the requirements.''',
    llm = my_llm,
    verbose=True,
)

In [20]:
frontend_engineer = Agent(
    role = 'A Gradio expert to who can write a simple frontend to demonstrate a backend',
    goal = '''Write a gradio UI that demonstrates the given backend, all in one file to be in the same directory as the backend module {module_name}.
    Here are the requirements: {requirements}''',
    backstory = '''You're a seasoned python engineer highly skilled at writing simple Gradio UIs for a backend class.
    You produce a simple gradio UI that demonstrates the given backend class; you write the gradio UI in a module app.py that is in the same directory as the backend module {module_name}.''',
    llm = my_llm,
    verbose=True,
)

In [21]:
test_engineer = Agent(
    role = 'An engineer with python coding skills who can write unit tests for the given backend module {module_name}',
    goal = '''Write unit tests for the given backend module {module_name} and create a test_{module_name} in the same directory as the backend module.''',
    backstory = '''You're a seasoned QA engineer and software developer who writes great unit tests for python code.''',
    llm = my_llm,
    verbose=True,
)

### Tasks to perform

In [25]:
# Create tasks for your agents
design_task = Task(
  description="""Write a python module that implements the design described by the engineering lead, in order to achieve the requirements.
    Here are the requirements: {requirements}""",
  agent=backend_engineer,
  verbose=True,
  expected_output="""A python module that implements the design and achieves the requirements.
    IMPORTANT: Output ONLY the raw Python code without any markdown formatting, code block delimiters, or backticks.
    The output should be valid Python code that can be directly saved to a file and executed."""

)

In [28]:
code_task = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.
  Your final answer MUST be the full blog post of at least 4 paragraphs.""",
  agent=backend_engineer,
  expected_output="An engaging blog post of at least 4 paragraphs highlighting significant AI advancements, written in an accessible and cool tone without complex AI jargon.",
  context=[design_task],
  output_file= "output/{module_name}"
)

In [29]:
frontend_task = Task(
  description="""Write a gradio UI in a module app.py that demonstrates the given backend class in {module_name}.
    Assume there is only 1 user, and keep the UI very simple indeed - just a prototype or demo.
    Here are the requirements: {requirements}""",
  agent=frontend_engineer,
  expected_output="""A gradio UI in module app.py that demonstrates the given backend class.
    The file should be ready so that it can be run as-is, in the same directory as the backend module, and it should import the backend class from {module_name}.
    IMPORTANT: Output ONLY the raw Python code without any markdown formatting, code block delimiters, or backticks.
    The output should be valid Python code that can be directly saved to a file and executed.""",
  context=[code_task],
  output_file= "output/app.py"
)

In [30]:
test_task = Task(
  description="""Write unit tests for the given backend module {module_name} and create a test_{module_name} in the same directory as the backend module.""",
  agent=test_engineer,
  expected_output="""A test_{module_name} module that tests the given backend module.
    IMPORTANT: Output ONLY the raw Python code without any markdown formatting, code block delimiters, or backticks.
    The output should be valid Python code that can be directly saved to a file and executed.""",
  context=[code_task],
  output_file= "output/test_{module_name}"
)

### Create a Crew

In [31]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[engineering_lead,backend_engineer,frontend_engineer,test_engineer],
  tasks=[design_task,code_task,frontend_task,test_task],
  verbose=False, # You can set it to 1 or 2 to different logging levels
)

In [32]:
crew

Crew(id=6a07d07c-d546-4ccb-9aad-4ec552f09c3b, process=Process.sequential, number_of_agents=4, number_of_tasks=4)

### Kickoff the crew - let the magic happen

In [36]:
requirements = """
A simple account management system for a trading simulation platform.
The system should allow users to create an account, deposit funds, and withdraw funds.
The system should allow users to record that they have bought or sold shares, providing a quantity.
The system should calculate the total value of the user's portfolio, and the profit or loss from the initial deposit.
The system should be able to report the holdings of the user at any point in time.
The system should be able to report the profit or loss of the user at any point in time.
The system should be able to list the transactions that the user has made over time.
The system should prevent the user from withdrawing funds that would leave them with a negative balance, or
 from buying more shares than they can afford, or selling shares that they don't have.
 The system has access to a function get_share_price(symbol) which returns the current price of a share, and includes a test implementation that returns fixed prices for AAPL, TSLA, GOOGL.
"""
module_name="accounts.py"
class_name = "Account"

template_input = {"requirements": requirements,"module_name" : module_name, "class_name" :class_name}



In [38]:
# Get your crew to work!
#result = crew.kickoff()
result = crew.kickoff(inputs=template_input)

# Agent: Python Engineer who can write code to achieve the design described by the engineering lead
## Task: Write a python module that implements the design described by the engineering lead, in order to achieve the requirements.
    Here are the requirements: 
A simple account management system for a trading simulation platform.
The system should allow users to create an account, deposit funds, and withdraw funds.
The system should allow users to record that they have bought or sold shares, providing a quantity.
The system should calculate the total value of the user's portfolio, and the profit or loss from the initial deposit.
The system should be able to report the holdings of the user at any point in time.
The system should be able to report the profit or loss of the user at any point in time.
The system should be able to list the transactions that the user has made over time.
The system should prevent the user from withdrawing funds that would leave them with a negative balance, 

In [39]:
print(result)

import unittest
import accounts

class TestAccounts(unittest.TestCase):

    def test_create_account(self):
        account = accounts.create_account("John Doe", "john.doe@example.com", "password123")
        self.assertIsNotNone(account)
        self.assertEqual(account["name"], "John Doe")
        self.assertEqual(account["email"], "john.doe@example.com")
        # Password verification is intentionally omitted for security reasons in this example.  In a real-world scenario, you would hash and securely compare passwords.


    def test_get_account(self):
        account = accounts.create_account("Jane Doe", "jane.doe@example.com", "securepass")
        retrieved_account = accounts.get_account(account["id"])
        self.assertEqual(retrieved_account, account)

    def test_get_account_not_found(self):
        retrieved_account = accounts.get_account(9999) #invalid ID
        self.assertIsNone(retrieved_account)

    def test_update_account(self):
      account = accounts.create_accou